In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import re
from urllib.parse import urljoin
import os
import pandas as pd
from sqlalchemy import create_engine, text

In [ ]:
# Para imprimir toda a informacción de celdas muy largas
pd.set_option('display.max_columns', 500)

# Para imprimir todas las líneas de un dataframe
pd.set_option('display.max_rows', None)

# Para mostrar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', 500)

In [3]:
# descargando los datos de Agenda de actividades y eventos de la API del Portal de datos abiertos del Ayuntamiento de Madrid

url = "https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.json"

try:
    response = requests.get(url)
    response.raise_for_status()
    raw_text = response.text
    cleaned_text = re.sub(r"\\(?![nrtbf\"\\/])", "", raw_text)
    data = json.loads(cleaned_text)

    if isinstance(data, dict):
        print(f"Top-level dictionary keys: {data.keys()}")
        if '@graph' in data and isinstance(data['@graph'], list):
            df = pd.DataFrame(data['@graph'])
            print(df.head())
        else:
            print("The list of events was not found within the '@graph' key.")
    else:
        print("Parsed JSON is not a dictionary as expected.")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Top-level dictionary keys: dict_keys(['@context', '@graph'])
                                                 @id  \
0  https://datos.madrid.es/egob/catalogo/tipo/eve...   
1  https://datos.madrid.es/egob/catalogo/tipo/eve...   
2  https://datos.madrid.es/egob/catalogo/tipo/eve...   
3  https://datos.madrid.es/egob/catalogo/tipo/eve...   
4  https://datos.madrid.es/egob/catalogo/tipo/eve...   

                                               @type        id  \
0  https://datos.madrid.es/egob/kos/actividades/C...  12734393   
1  https://datos.madrid.es/egob/kos/actividades/T...  12749968   
2  https://datos.madrid.es/egob/kos/actividades/M...  12767335   
3  https://datos.madrid.es/egob/kos/actividades/P...  12758961   
4  https://datos.madrid.es/egob/kos/actividades/C...  12747998   

                                               title  \
0                         2º encuentro 'Estuve allí'   
1  3º Festival de Artes Escénicas para la primera...   
2                            5 estrel

In [ ]:
url = "https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.json"

try:
    response = requests.get(url)
    response.raise_for_status()
    raw_text = response.text
    cleaned_text = re.sub(r"\\(?![nrtbf\"\\/])", "", raw_text)
    data = json.loads(cleaned_text)

    if isinstance(data, dict):
        print(f"Top-level dictionary keys: {data.keys()}")
        if '@graph' in data and isinstance(data['@graph'], list):
            df = pd.DataFrame(data['@graph'])
            print(df.head())
        else:
            print("The list of events was not found within the '@graph' key.")
            df = pd.DataFrame() # Inicializar un DataFrame vacío
    else:
        print("Parsed JSON is not a dictionary as expected.")
        df = pd.DataFrame() # Inicializar un DataFrame vacío

except requests.exceptions.RequestException as e:
    print(f"Error fetching data from Ayuntamiento de Madrid: {e}")
    df = pd.DataFrame() # Inicializar un DataFrame vacío
except json.JSONDecodeError as e:
    print(f"Error decoding JSON from Ayuntamiento de Madrid: {e}")
    df = pd.DataFrame() # Inicializar un DataFrame vacío
except Exception as e:
    print(f"An unexpected error occurred while fetching Ayuntamiento de Madrid data: {e}")
    df = pd.DataFrame() # Inicializar un DataFrame vacío

if not df.empty:
    # Haciendo el filtrado del dataset
    filtered_df = df[df['audience'].isin(['Familias', 'Niños', 'Jovenes,Niños', 'Niños,Familias', 'Jovenes,Niños,Familias'])].copy() # utilizar .copy() para evitar SettingWithCopyWarning

    extracted_data = []
    for index, row in filtered_df.iterrows():
        extracted_event = {
            "name": row["title"],
            "url": row["link"],
            "image": None,
            "start_time": row["time"],
            "start_date": pd.to_datetime(row["dtstart"]).date() if pd.notna(row["dtstart"]) else None,
            "end_time": None,
            "end_date": pd.to_datetime(row["dtend"]).date() if pd.notna(row["dtend"]) else None,
            "duration": None,
            "venue_name": row["event-location"],
            "venue_address": row["address"].get("area", {}).get("street-address")
                                    if isinstance(row["address"], dict) and isinstance(row["address"].get("area"), dict) else None,
            "longitude": row["location"].get("longitude")
                                    if isinstance(row["location"], dict) and row["location"].get("longitude") else None, # Comprueba si la ubicación es un diccionario
            "latitude": row["location"].get("latitude")
                                    if isinstance(row["location"], dict) and row["location"].get("latitude") else None, # Comprueba si la ubicación es un diccionario
            "summary": row["description"],
            "is_online_event": None,
            "tickets_url": row["link"],
            "tags": [row["audience"]] if pd.notna(row["audience"]) else [],
            "event_id": row["uid"],
            "data_source": "ayuntamiento madrid"
        }
        extracted_data.append(extracted_event)

    ayuntamiento_madrid_df = pd.DataFrame(extracted_data)
    print(ayuntamiento_madrid_df.head())
    print(ayuntamiento_madrid_df.info())


# Función para extraer la URL de la imagen de cada evento y almacenarla
    base_url = "https://www.madrid.es"

    def get_event_image(event_url):
        try:
            response = requests.get(event_url, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")

            img_div = soup.find("div", class_="image-content ic-right")
            if img_div and img_div.img:
                img_src = img_div.img.get("src")
                if img_src:
                    full_img_url = urljoin(base_url, img_src)
                    return full_img_url
        except Exception as e:
            print(f"Error scraping image from {event_url}: {e}")
        return None

    ayuntamiento_madrid_df["image"] = ayuntamiento_madrid_df["url"].apply(get_event_image)
else:
    ayuntamiento_madrid_df = pd.DataFrame()

Top-level dictionary keys: dict_keys(['@context', '@graph'])
                                                 @id  \
0  https://datos.madrid.es/egob/catalogo/tipo/eve...   
1  https://datos.madrid.es/egob/catalogo/tipo/eve...   
2  https://datos.madrid.es/egob/catalogo/tipo/eve...   
3  https://datos.madrid.es/egob/catalogo/tipo/eve...   
4  https://datos.madrid.es/egob/catalogo/tipo/eve...   

                                               @type        id  \
0  https://datos.madrid.es/egob/kos/actividades/C...  12734393   
1  https://datos.madrid.es/egob/kos/actividades/T...  12749968   
2  https://datos.madrid.es/egob/kos/actividades/M...  12767335   
3  https://datos.madrid.es/egob/kos/actividades/P...  12758961   
4  https://datos.madrid.es/egob/kos/actividades/C...  12747998   

                                               title  \
0                         2º encuentro 'Estuve allí'   
1  3º Festival de Artes Escénicas para la primera...   
2                            5 estrel

In [11]:
ayuntamiento_madrid_df

,name,url,image,start_time,start_date,end_time,end_date,duration,venue_name,venue_address,longitude,latitude,summary,is_online_event,tickets_url,tags,event_id,data_source
0,3º Festival de Artes Escénicas para la primera infancia. Espuma de mar,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=a084e7802e416910VgnVCM2000001f4a900aRCRD,https://www.madrid.es/UnidadWeb/UGBBDD/Actividades/Distritos/Hortaleza/Actividades%20CC/Eventos/ficheros/espuma.jpg,,2025-05-18,None,2025-05-18,None,Centro Cultural Sanchinarro (Hortaleza),CALLE PRINCESA DE EBOLI 29,-3.655347,40.490422,"Compañía Titirisolfa. Edad recomendada: de 0 meses a 5 años. Dos sesiones: 11 y 12:30 horas. Entrada: 8 euros niño y 8 euros adulto. (Niño y adulto tiene que disponer de entrada). Aforo: 40 niños, 40 acompañantes, que subirán al escenario. El resto de acompañantes deberán visionar el espectáculo desde el patio de butacas con entrada de patio de butacas. Duración: 50 minutos. ¿Te imaginas que un día te despiertas y ha crecido una playa en el teatro? Una playa, una bañista, unmarinero y una re...",None,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=a084e7802e416910VgnVCM2000001f4a900aRCRD,"[Niños,Familias]",12749968,ayuntamiento madrid
1,A la mar salada,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=5a176267aec56910VgnVCM1000001d4a900aRCRD,https://www.madrid.es/UnidadesDescentralizadas/Bibliotecas/BibliotecasPublicas/Actividades/Actividades_Infantiles_Juveniles/Cuentacuentos/ficheros/PiernasPequeConLibro_260.png,18:00,2025-05-22,None,2025-05-22,None,Biblioteca Pública Municipal Gerardo Diego (Villa de Vallecas),CALLE MONTE AYA 12,-3.622730,40.377902,"Duendes, tesoros, aventuras, misterios... Las bibliotecas están llenas de historias que cuenteros y cuenteras nos acercan a través de su creatividad y su magia. Este cuento está pensado para niñas y niños a partir de 2 años. Recuerda que las niñas y niños deberán ir acompañados por una persona adulta. Entrada libre hasta completar aforo. Te esperamos en la biblioteca para pasar un buen rato lleno de fantasía, de la mano de Margalida Alberti",None,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=5a176267aec56910VgnVCM1000001d4a900aRCRD,[Niños],12758039,ayuntamiento madrid
2,A taste of the moon,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=baceaa4735436910VgnVCM1000001d4a900aRCRD,https://www.madrid.es/UnidadesDescentralizadas/Bibliotecas/BibliotecasPublicas/Actividades/Actividades_Infantiles_Juveniles/Cuentacuentos/ficheros/Cuentacuentos-joven-leyendo_260.png,18:00,2025-05-08,None,2025-05-08,None,Biblioteca Pública Municipal Vicálvaro,CALLE VILLARDONDIEGO 36,-3.608628,40.405284,"Vamos a leer en inglés y a divertirnos en la biblioteca. Duendes, tesoros, aventuras, misterios... Las bibliotecas están llenas de historias que cuenteros y cuenteras nos acercan a través de su creatividad y su magia. Este cuento está pensado para niñas y niños a partir de 2 años. Para asistir es necesaria la inscripción a través de este enlace . La biblioteca se pondrá en contacto contigo para confirmar tu asistencia. Recuerda que los niños y niñas deberán ir acompañados por una perso...",None,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=baceaa4735436910VgnVCM1000001d4a900aRCRD,[Niños],12754392,ayuntamiento madrid
3,A taste of the moon,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=edd0528353436910VgnVCM2000001f4a900aRCRD,https://www.madrid.es/UnidadesDescentralizadas/Bibliotecas/BibliotecasPublicas/Actividades/Actividades_Infantiles_Juveniles/Cuentacuentos/ficheros/230425LeonFeliz_260.png,17:30,2025-05-22,None,2025-05-22,None,Biblioteca Pública Municipal Pab

In [ ]:
# Variable para conexíon a la base de datos

RENDER_DB_URI = os.getenv("RENDER_DB_URI")

TABLE_NAME = 'all_events'
UNIQUE_ID_COLUMN = 'event_id'

# Creando motor SQLAlchemy
engine = create_engine(RENDER_DB_URI)

# Cargar columnas event_id y imagen existentes desde la base de datos
query = f"SELECT {UNIQUE_ID_COLUMN}, image FROM {TABLE_NAME}"
db_df = pd.read_sql(query, con=engine)

# Unir con datos del ayuntamiento_madrid_df para comparar imágenes
merged_df = pd.merge(
    ayuntamiento_madrid_df[[UNIQUE_ID_COLUMN, 'image']],
    db_df,
    on=UNIQUE_ID_COLUMN,
    how='left',
    suffixes=('_new', '_old')
)

# Identificar filas donde la imagen es diferente o anteriormente era NULA
to_update = merged_df[merged_df['image_new'].notna()]

# Actualizar la columna de imagen en la base de datos
with engine.begin() as conn:
    for _, row in to_update.iterrows():
        update_stmt = text(f"""
            UPDATE {TABLE_NAME}
            SET image = :image
            WHERE {UNIQUE_ID_COLUMN} = :event_id
        """)
        conn.execute(update_stmt, {
            "image": row['image_new'],
            "event_id": row[UNIQUE_ID_COLUMN]
        })

print(f"✅ Updated {len(to_update)} records in the database.")


✅ Updated 265 records in the database.


Cargar un df para comprobar los eventos con un filtrado

In [ ]:
check_df = pd.read_csv('../data/raw/from_render_complete_db_BKP_IMGS.csv')

In [ ]:
check_df[check_df['data_source']=='ayuntamiento madrid']

,event_id,name,url,start_time,start_date,end_time,end_date,duration,venue_name,venue_address,...,latitude,summary,is_online_event,tickets_url,tags,data_source,image,age_range,min_age,max_age
693,12762483,"CICLO BEBÉS. Teatro Tyl Tyl, presenta 'Tylito'",http://www.madrid.es/sites/v/index.jsp?vgnextc...,11:00,2025-05-10,NaN,2025-05-10,NaN,Centro Cultural Príncipe de Asturias (Ciudad L...,CALLE HERMANOS GARCIA NOBLEJAS 14,...,40.436447,Concierto teatralizado para un público activo ...,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Cine,ayuntamiento madrid,https://res.cloudinary.com/doll2vtjc/image/upl...,0-3,0.0,3.0
694,12718157,Cine infantil en la Biblioteca María Lejárraga...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,18:00,2025-05-13,NaN,2025-05-13,NaN,Biblioteca Pública Municipal María Lejárraga (...,CALLE PRINCESA DE EBOLI 29,...,40.490416,Ven a la biblioteca a disfrutar del cine. Ve...,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Cine,ayuntamiento madrid,https://res.cloudinary.com/doll2vtjc/image/upl...,NaN,NaN,NaN
695,12723003,Érase una vez un pato - Teatro La Proa (Cuba),http://www.madrid.es/sites/v/index.jsp?vgnextc...,18:00,2025-05-03,NaN,2025-05-04,NaN,Teatro Municipal de Títeres. Parque de El Retiro,AVENIDA MÉXICO 4,...,40.418784,NaN,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Cine,ayuntamiento madrid,https://res.cloudinary.com/doll2vtjc/image/upl...,NaN,NaN,NaN
696,12762496,MÚSICA. La pequeña orquesta cinematográfica.,http://www.madrid.es/sites/v/index.jsp?vgnextc...,19:00,2025-05-17,NaN,2025-05-17,NaN,Centro Cultural Príncipe de Asturias (Ciudad L...,CALLE HERMANOS GARCIA NOBLEJAS 14,...,40.436447,Un espectáculo músico-teatral que te transport...,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Cine,ayuntamiento madrid,https://res.cloudinary.com/doll2vtjc/image/upl...,NaN,NaN,NaN
697,12765900,Taller de teatro infantil,http://www.madrid.es/sites/v/index.jsp?vgnextc...,NaN,2025-05-10,NaN,2025-05-31,NaN,Centro Cultural Carril del Conde (Hortaleza),CALLE CARRIL DEL CONDE 57,...,40.460895,NaN,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Cine,ayuntamiento madrid,https://res.cloudinary.com/doll2vtjc/image/upl...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,12759320,¡Soy el chico/a del tiempo!,http://www.madrid.es/sites/v/index.jsp?vgnextc...,12:00,2025-05-31,NaN,2025-05-31,NaN,Centro de Educación Ambiental y Cultural Maris...,CALLE DOCTOR TOLOSA LATOUR 16,...,40.374166,Horario: ﻿de 12 a 13 horas. Edad: Familias co...,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Ocio,ayuntamiento madrid,https://res.cloudinary.com/doll2vtjc/image/upl...,+6,6.0,14.0
1022,12751238,¿Hay alguien más aburrido que yo?,http://www.madrid.es/sites/v/index.jsp?vgnextc...,18:00,2025-05-09,NaN,2025-05-09,NaN,Biblioteca Pública Municipal Pablo Neruda (Ciu...,CALLE ASCAO 4,...,40.429411,Una historia divertida y tierna sobre el poder...,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Ocio,ayuntamiento madrid,https://res.cloudinary.com/doll2vtjc/image/upl...,+6,6.0,14.0
1023,12762235,¿Jugamos con Europa...?,http://www.madrid.es/sites/v/index.jsp?vgnextc...,18:00,2025-05-08,NaN,2025-05-08,NaN,Biblioteca Pública Municipal La Chata (Caraban...,CALLE GENERAL RICARDOS 252,...,40.383306,"La biblioteca celebra el Día de Europa, jugand...",NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Ocio,ayuntamiento madrid,https://res.cloudinary.com/doll2vtjc/image/upl...,+8,8.0,14.0
1024,12762621,¿Qué hemos hecho en Madrid?,http://www.madrid.es/sites/v/index.jsp?vgnextc...,18:00,2025-05-17,NaN,2025-05-17,NaN,Parque Santa Eugenia,CALLE Cerro Almodóvar 1,...,40.381311,Compañía Nos separa el puente y el río Ub...,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Ocio,ayuntamiento madrid,https://res.cloudinary.com/doll2vtjc/image/upl...,6-12,6.0,12.0


In [ ]:
# Función auxiliar para actualizar DB

# Cargando variable de entorno
RENDER_DB_URI = os.getenv("RENDER_DB_URI")
TABLE_NAME = 'all_events'
UNIQUE_ID_COLUMN = 'event_id'

# Your DataFrame (example)
# df = pd.DataFrame({
#     'event_id': [1, 2],
#     'image': ['img1.png', 'img2.png']
# })

# Creando motor SQLAlchemy
engine = create_engine(RENDER_DB_URI)

# Abrindo la conexión
with engine.begin() as conn:
    for _, row in df.iterrows():
        query = text(f"""
            UPDATE {TABLE_NAME}
            SET image = :image
            WHERE {UNIQUE_ID_COLUMN} = :event_id
        """)
        conn.execute(query, {
            "image": row["image"],
            "event_id": str(row["event_id"])  # ✅ Para garantizar que sea tipos tring
        })

print("Database updated successfully.")


Database updated successfully.
